In [ ]:
import os
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import os


K.set_image_dim_ordering('th')

#Preprocessing Module

train_path=r'C:\Users\Lenovo\Desktop\mlmp\Images\train'
test_path=r'C:\Users\Lenovo\Desktop\mlmp\Images\test'
valid_path=r'C:\Users\Lenovo\Desktop\mlmp\Images\valid'

#train_path=r'C:\Users\Lenovo\Desktop\mlmp\salman1\train'
#test_path=r'C:\Users\Lenovo\Desktop\mlmp\salman1\test'
#valid_path=r'C:\Users\Lenovo\Desktop\mlmp\salman1\valid'

train_datagen=ImageDataGenerator()



train_batches=train_datagen.flow_from_directory(train_path,target_size=(150,150),classes=None,color_mode="rgb",class_mode='categorical',batch_size=10)
test_batches=  train_datagen.flow_from_directory(test_path,target_size=(150,150),classes=None,color_mode="rgb",class_mode='categorical',batch_size=10)
valid_batches= train_datagen.flow_from_directory(valid_path,target_size=(150,150),classes=None,color_mode="rgb",class_mode='categorical',batch_size=10)


def plots(imgs, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(imgs[0]) is np.ndarray:
        ims = np.array(imgs).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
            
        
        plt.imshow(ims[i], interpolation=None if interp else 'none')
imgs,labels=next(train_batches)
plots(imgs,titles=labels)


#test_imgs,test_labels= next (test_batches)
#plots(test_imgs,titles=test_labels)


#test_imgs,test_labels= next (valid_batches)
#plots(test_imgs,titles=test_labels)

#Build and Train CNN
model=Sequential([
       
    Conv2D(32,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    Conv2D(32,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None),
    Conv2D(64,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    Conv2D(64,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None),
    Conv2D(128,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    Conv2D(128,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None),
    Conv2D(256,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    Conv2D(256,(3,3),activation='relu',input_shape=(3,150,150),padding="same"),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None),
    
    
    
    
    Flatten(),
    Dense(2,activation='softmax'),
    
    
    
    
    
])

#model.summary();
model.compile(Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_batches,steps_per_epoch=50,validation_data=valid_batches,validation_steps=4,epochs=20,verbose=2)

test_imgs,test_labels=next(test_batches)
plots(test_imgs,titles=test_labels)
test_labels=test_labels[:,0]
test_labels

predictions=model.predict_generator(test_batches,steps=1,verbose=0)
cm=confusion_matrix(test_labels,np.round(predictions[:,0]))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    # Only use the labels that appear in the data
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh=cm.max()/2
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,cm[i,j],horizontalalignment="center",color="white" if cm[i,j]>thresh else "black")
        
    plt.tight_layout()
    plt.xlabel('Predicted Label')
    plt.ylabel('True label')
cm_plot_labels=['male','female']
plot_confusion_matrix(cm,cm_plot_labels,title='Confusion Matrix')